In [3]:
import numpy as np
from tqdm.notebook import tqdm

In [6]:
# CFAR version 2, in this slidin window is created 
#on the basis of value of the pixel

class CFAR_v2(object):

    #initializing the values
    
    def __init__(self,img,tw,gw,bw,pfa):
        self.img = img
        self.tw = tw
        self.gw = gw
        self.bw = bw
        self.pfa = pfa
        self.dvi = []
        self.noise_data = []
        self.final_image = []
        #print("Kernel Ready.")
           
    #checking if the pixel exists
    def isPixelexists(self,size_img,a,b):
        r,c = size_img
        #print(r,c)
        if (a>=0 and a<r) and (b>=0 and b<c) :
            return True
        else:
            return False

    #Computing 4 buffer values.TOP,BOTTOM,LEFT and RIGHT
    def get_topBuffer(self,u,v,size_t,size_g):
        top_buffer = []

        #we have considered the target_window pixels too.
        for p in range(size_t,size_g+1):
                        
            x = u-p
            for m in range(-p,p+1):
                y = v+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    top_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    top_buffer.append(0)

        return top_buffer

    def get_bottomBuffer(self,u,v,size_t,size_g):
        bottom_buffer = []
        
        for p in range(size_t,size_g+1):
            
            x = u+p
            for m in range(-p,p+1):
                y = v+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    bottom_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    bottom_buffer.append(0)

        return bottom_buffer

    def get_leftBuffer(self,u,v,size_t, size_g):
        left_buffer = []
        for p in range(size_t,size_g+1):
            y = v-p
            for m in range(-p,p+1):
                x = u+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    left_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    left_buffer.append(0)

        return left_buffer

    def get_rightBuffer(self,u,v,size_t,size_g):
        right_buffer = []
        
        for p in range(size_t,size_g+1):
            y = v+p
            for m in range(-p,p+1):
                x = u+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    right_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    right_buffer.append(0)

        return right_buffer

    
    def computeDVnThreshold(self):
        #print("Computing DVi and Threshold..")
        
        radius_t = int(self.tw/2)
        radius_g = int(self.bw/2)
        rows = self.img.shape[0]
        cols = self.img.shape[1]
        
        for d in (range(0, rows * cols)):
     
            # Find row and column index
            i = d // cols
            j = d % cols
                
            if self.img[i,j] != 0:
                #print("hello")
                win_top_buffer = self.get_topBuffer(i,j,radius_t,radius_g)
                win_bottom_buffer = self.get_bottomBuffer(i,j,radius_t,radius_g)
                win_left_buffer = self.get_leftBuffer(i,j,radius_t,radius_g)
                win_right_buffer = self.get_rightBuffer(i,j,radius_t,radius_g)

                guard_buffer = np.array(

                    [win_top_buffer,win_bottom_buffer,win_left_buffer,win_right_buffer]

                )

                self.dvi.append((self.img[i][j] - guard_buffer.mean())/guard_buffer.std())
                self.noise_data.append((guard_buffer.mean()))
            else:
                self.dvi.append(0)
                self.noise_data.append(0)

        self.dvi = np.array(self.dvi).reshape(self.img.shape)
        self.noise_data = (np.array(self.noise_data))
        #P = np.array(self.compute_scaleFactor()*noise_data).reshape(self.img.shape)
        #print("Process completed, DV image and Threshold Image succesfully Computed.\n")
        return self.dvi,self.noise_data

#     def compute_noise(self):
#         noise_data = []
#         radius_t = int(self.tw/2)
#         radius_g = int(self.bw/2)
#         print("Computing P...")

#         for i in tqdm(range(self.img.shape[0])):
#             for j in range(self.img.shape[1]):
                
#                 win_top_buffer = self.get_topBuffer(i,j,radius_t,radius_g)
#                 win_bottom_buffer = self.get_bottomBuffer(i,j,radius_t,radius_g)
#                 win_left_buffer = self.get_leftBuffer(i,j,radius_t,radius_g)
#                 win_right_buffer = self.get_rightBuffer(i,j,radius_t,radius_g)

#                 background_buffer = np.array(

#                     [win_top_buffer,win_bottom_buffer,win_left_buffer,win_right_buffer]

#                 )

#                 #print(guard_buffer)
#                 #print(guard_buffer.mean())
#                 noise_data.append(float(background_buffer.mean()))
                
#         noise_data = (np.array(noise_data))
#         #print(noise_data)
#         P = np.array(self.compute_scaleFactor()*noise_data).reshape(self.img.shape)
#         print("Process Completed, P image succesfully computed.\n")
#         return P
    
    def compute_scaleFactor(self):
#         N = 0
#         for b in range(self.tw,self.bw+1):
#             if b%2 != 0:
#                 N += 4*b -4
#         print(N)
        #return (N*(self.pfa**(-1/N) -1))
        return self.pfa
    
    def shipDetection(self):
               
        DV,T = self.computeDVnThreshold()
        #T = self.pfa
        
        for i in range(self.img.shape[0]):
            for j in range(self.img.shape[1]):
                
                if DV[i][j] > T[i][j]:
                    
                    self.final_image.append(1)
                else:
                    self.final_image.append(0) #valid Ships
        
        self.final_image = np.array(self.final_image).reshape(self.img.shape)
        #print("Binary Image of Ships is Succesfully Generated.\n")
        return final_image
    
    def computeN(self):
        N = 0
        for b in range(self.tw,self.bw+1):
            if b%2 != 0:
                N += 4*b -4
        return N